In [9]:
# Включване на нужните библиотеки
# Including libraries

import random
import sys
import io
import numpy as np
from keras.models import model_from_json

In [10]:
# Дефинираме нужни методи
# Defining some useful methods

def loadModel():
    jsonFile = open('model/modelSmqx.json', 'r')
    loadedModelJSON = jsonFile.read()
    jsonFile.close()
    model = model_from_json(loadedModelJSON)
    model.load_weights("model/modelSmqx.h5")
    return model


def loadText():
    text = open("datasets/chalga.txt").read().decode('utf-8')
    chars = sorted(list(set(text)))
    charIndices = dict((c, i) for i, c in enumerate(chars))
    indicesChar = dict((i, c) for i, c in enumerate(chars))
    return text, chars, charIndices, indicesChar

In [11]:
# Зареждаме модела и някои променливи, които са нужни за неговата работа
# Loading the model and some variables that are necessary for the model's work

model = loadModel()
text, chars, charIndices, indicesChar = loadText()
chunkSize = 56

In [12]:
# Поглед върху структурата на модела
# Quick peek in the model's structure

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 148)               166352    
_________________________________________________________________
dense_1 (Dense)              (None, 132)               19668     
_________________________________________________________________
activation_1 (Activation)    (None, 132)               0         
Total params: 186,020
Trainable params: 186,020
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Дефинираме функцията "sample", която връща предсказание за следващ символ
# Defining the function "sample" which is generating the probability of each next character


# Вход: лист от всички символи; Изход: най-вероятния следващ символ
# Input: list of all symbols; Output: the character with highest probability of being the next character

def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def generateText(startingText):
    start_index = random.randint(0, len(text) - chunkSize - 1)
    
    generated = ''
    sentence = text[start_index: start_index + chunkSize]
        #text[start_index: start_index + chunkSize]
        #startingText.decode('utf-8')
    generated += sentence
    
    print("Generating text with seed: \"" + sentence + '"\n')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, chunkSize, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, charIndices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = indicesChar[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    print('\n')

In [15]:
# Генериране на текст с начален примерен текст 
# Generate text with sample starting sentence 

generateText("Направи ме дива и щастлива, обичай ме")

Generating text with seed: "ш безброй жени, но си виновен, зная ще страдаш ти. Пак щ"

ш безброй жени, но си виновен, зная ще страдаш ти. Пак ще градиш ляна на сувсии само за мен нче. ясно, викти, кажи, щом ще повярвай морчя няма с твоите - плая за дени знам.
Сялки дочи. Бох ускижая ли - правя кога ще с л-че. На мене - париш от доза край терка, тора рана! Слеловелна, всичко спишал, какво ми е е.  А. Ти ха гочи момче - колко става маза, ждах ме, ще стана антириш ня всичко ще гори да болтол.. жени, усествай, повтаряй на кой ли лоцита и мъж()


